<a href="https://colab.research.google.com/github/davrodrod/algorirmosIA/blob/master/RNN_LTSM_TimeSeriesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Networks

Este tipo de redes están diseñadas para operar sobre secuencias de datos

Han probado ser muy efectivas para Procesado de Lenguaje Natural donde las secuncias de texto son las entradas. También han tenido algo de éxito en previsión de series de tiempo y en reconocimiento de habla.

El tipo más popular de RNN es el LSTM (Long Short-Term Memory Network). Éstas pueden utilizarse para aceptar una secuencia de entradas y hacer una predicción, como una asignación a una clase o una predicción numérica como el siguiente valor en la secuencia.

Se utiliza un dataset de venta de coches para predicciones univariable. El objetivo es predecir el número de ventas de coches por mes. 

Se utiliza una ventana de los últimos 5 meses para predecir los resultados del mes actual (función split_sequence())

Por ejemplo, si la secuencia es: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
las muestras para entrenamiento serán:
Input: 1, 2, 3, 4, 5   -> Output: 6
Input: 2, 3, 4, 5, 6   -> Output: 7
Input: 3, 4, 5, 6, 7   -> Output: 8
...

Dado que es un problema de regresión se utiliza una función de activación lineal (o lo que es lo mismo, no hay función de activación), y se optimiza el MSE. Tb se evalúa con el MAE



In [1]:
# Fuente https://machinelearningmastery.com/tensorflow-tutorial-deep-learning-with-tf-keras/

# lstm for time series forecasting
from numpy import sqrt
from numpy import asarray
from pandas import read_csv
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
        X, y = list(), list()
        for i in range(len(sequence)):
                # find the end of this pattern
                end_ix = i + n_steps
                # check if we are beyond the sequence
                if end_ix > len(sequence)-1:
                        break
                # gather input and output parts of the pattern
                seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                X.append(seq_x)
                y.append(seq_y)
        return asarray(X), asarray(y)

# load the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv'
df = read_csv(path, header=0, index_col=0, squeeze=True)
# retrieve the values
values = df.values.astype('float32')
# specify the window size
n_steps = 5
# split into samples
X, y = split_sequence(values, n_steps)
print(X)
print(y)
# reshape into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))
print(X)
# split into train/test
n_test = 12
X_train, X_test, y_train, y_test = X[:-n_test], X[-n_test:], y[:-n_test], y[-n_test:]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', kernel_initializer='he_normal', input_shape=(n_steps,1)))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1))
# compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# fit the model
model.fit(X_train, y_train, epochs=350, batch_size=32, verbose=2, validation_data=(X_test, y_test))
# evaluate the model
mse, mae = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % (mse, sqrt(mse), mae))
# make a prediction
row = asarray([18024.0, 16722.0, 14385.0, 21342.0, 17180.0]).reshape((1, n_steps, 1))
yhat = model.predict(row)
print('Predicted: %.3f' % (yhat))

[[ 6550.  8728. 12026. 14395. 14587.]
 [ 8728. 12026. 14395. 14587. 13791.]
 [12026. 14395. 14587. 13791.  9498.]
 [14395. 14587. 13791.  9498.  8251.]
 [14587. 13791.  9498.  8251.  7049.]
 [13791.  9498.  8251.  7049.  9545.]
 [ 9498.  8251.  7049.  9545.  9364.]
 [ 8251.  7049.  9545.  9364.  8456.]
 [ 7049.  9545.  9364.  8456.  7237.]
 [ 9545.  9364.  8456.  7237.  9374.]
 [ 9364.  8456.  7237.  9374. 11837.]
 [ 8456.  7237.  9374. 11837. 13784.]
 [ 7237.  9374. 11837. 13784. 15926.]
 [ 9374. 11837. 13784. 15926. 13821.]
 [11837. 13784. 15926. 13821. 11143.]
 [13784. 15926. 13821. 11143.  7975.]
 [15926. 13821. 11143.  7975.  7610.]
 [13821. 11143.  7975.  7610. 10015.]
 [11143.  7975.  7610. 10015. 12759.]
 [ 7975.  7610. 10015. 12759.  8816.]
 [ 7610. 10015. 12759.  8816. 10677.]
 [10015. 12759.  8816. 10677. 10947.]
 [12759.  8816. 10677. 10947. 15200.]
 [ 8816. 10677. 10947. 15200. 17010.]
 [10677. 10947. 15200. 17010. 20900.]
 [10947. 15200. 17010. 20900. 16205.]
 [15200. 170